In [76]:
import pandas as pd

In [77]:
# read data
data = pd.read_csv('data.csv', sep='delimiter')

/tmp/ipykernel_1520/2753078877.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('data.csv', sep='delimiter')


In [78]:
# Data preprocessing
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import spellchecker
import string

stop_words = set(stopwords.words('english')) # set of stopwords
exclude = set(string.punctuation) # set of all special characters
lemma = WordNetLemmatizer() # lemmatizer

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop_words])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

preprocessed_data = pd.DataFrame(data.text.apply(clean))

In [79]:
arr = []
np_arr = preprocessed_data.to_numpy()

for e in np_arr:
    arr.append(e[0])

arr

['hmmm exactly month since scammed still remember exactly 3pm friday walking around shopping mall a girl sent message fb wanted chat me met girl dating site dateinasia specific found actually facebook account site added surprised knew fb kept asking personal thing like do parent do tried find financial support right first time chati knownthat week scami went back shopping mall started chatting her while changed skype chatshe asked alone felt boredand asked video chat said would take clothes same think much attractive girl cam got exposedshe recorded sent youtube link video link friend facebook told want friend see video send money otherwise send friend facebook scared panic completely controlthat pathetic moment felt stupidi lot beggingshe said send money it said always keep word choice believe thatand needed immediately ran western union 535pm send money staff told system close 530pm 5 minute late tried another money shop same send immediately told reason gave another chance send mone

In [80]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=10)
X = vectorizer.fit_transform(arr)

feature_names = vectorizer.get_feature_names_out()
feature_names

array(['believe', 'could', 'day', 'even', 'friend', 'get', 'got', 'it',
       'like', 'me', 'money', 'much', 'one', 'really', 'said', 'scam',
       'still', 'time', 'told', 'want', 'would'], dtype=object)

In [81]:
from sklearn.decomposition import LatentDirichletAllocation

number_of_topics = 3

model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)

model.fit(X.toarray())

LatentDirichletAllocation(n_components=3, random_state=0)

In [82]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

display_topics(model, feature_names, 15)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights
0,would,47.9,still,0.3,money,65.3
1,one,32.5,even,0.3,said,45.3
2,day,22.0,like,0.3,me,42.3
3,like,19.5,could,0.3,told,42.2
4,much,17.4,really,0.3,friend,31.6
5,time,15.4,time,0.3,got,25.2
6,scam,13.7,want,0.3,get,24.3
7,money,12.3,much,0.3,want,21.6
8,said,11.4,get,0.3,scam,21.0
9,could,11.1,day,0.3,it,20.1
